In [1]:
import torch
import numpy as np
import cv2
from random import randrange
import gc

In [10]:
def frame_swap(video):
    idx = [randrange(0, 300, 2) for p in range(60)]
    swapped_video = video.clone()
  
    for i in idx:
        swapped_video[i], swapped_video[i+1] = video[i+1], video[i]
    return swapped_video

In [4]:
def tensor_to_video(hsv_tensor, output_path="output.mp4", fps=60):

    num_frames, _, height, width = hsv_tensor.shape
    fourcc = cv2.VideoWriter_fourcc(*'x264')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame_idx in range(num_frames):
        hsv_frame = hsv_tensor[frame_idx].cpu().numpy().astype(np.uint8)  # Convert tensor to NumPy
        hsv_frame = np.transpose(hsv_frame, (1, 2, 0))
        # Convert HSV back to BGR for saving
        bgr_frame = cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)

        out.write(bgr_frame)

    out.release()

In [5]:
def iso_val(video):
    val_tensor = []
    for i, frame in enumerate(video):
        frame = torch.reshape(frame[2], (1, frame.shape[-2], frame.shape[-1]))
        val_tensor.append(frame)
        
    val_tensor = frames = torch.stack(val_tensor)

    return val_tensor

In [8]:
names = ["rush1", "sintel3"] #"aspen", "crowd2", "ducks", "life1", "pedestrian2", "sunflower1", 

In [9]:
for name in names:
    hsv_tensor = torch.load(f"{name}_hsv_tensor", weights_only=True)
    swapped_tensor = frame_swap(hsv_tensor).detach()
    #torch.save(swapped_tensor, f"{name}_swapped_tensor")
    tensor_to_video(swapped_tensor, f'{name}_0419_swapped.mp4')
    
    swapped_val_tensor = iso_val(swapped_tensor).detach()
    torch.save(swapped_val_tensor, f"{name}_swapped_val_tensor")
    
    del hsv_tensor, swapped_tensor, swapped_val_tensor
    
    print(f'{name} done')
    gc.collect()

OpenCV: FFMPEG: tag 0x34363278/'x264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


rush1 done


OpenCV: FFMPEG: tag 0x34363278/'x264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


sintel3 done
